# Notebook 12: Working with Physical Units

Underworld3 supports physical units throughout the modeling workflow. This notebook shows you how to:

- Create quantities with units (velocities, temperatures, viscosities)
- Create meshes with coordinate units (meters, kilometers)
- Work with mesh coordinates that have units
- Extract raw numerical values when needed

The units system helps ensure dimensional consistency and makes your models easier to understand.

In [1]:
import nest_asyncio
nest_asyncio.apply()

import underworld3 as uw
import numpy as np

## Creating Physical Quantities

Physical quantities are created using `uw.units` (which provides access to the Pint units library):

In [2]:
# Create quantities with units
mantle_depth = 2900 * uw.units.km
plate_velocity = 5 * uw.units.cm / uw.units.year
mantle_temperature = 1500 * uw.units.K
mantle_viscosity = 1e21 * uw.units.Pa * uw.units.s

print(f"Depth: {mantle_depth}")
print(f"Velocity: {plate_velocity}")
print(f"Temperature: {mantle_temperature}")
print(f"Viscosity: {mantle_viscosity}")

Depth: 2900 kilometer
Velocity: 5.0 centimeter / year
Temperature: 1500 kelvin
Viscosity: 1e+21 pascal * second


### Unit Conversions

You can convert between compatible units:

In [3]:
# Convert units
print(f"Velocity in mm/year: {plate_velocity.to('mm/year')}")
print(f"Depth in meters: {mantle_depth.to('meter')}")
print(f"Viscosity: {mantle_viscosity.to('Pa * s')}")

Velocity in mm/year: 50.0 millimeter / year
Depth in meters: 2900000.0 meter
Viscosity: 1e+21 pascal * second


## Meshes with Coordinate Units

Meshes can have physical coordinate units. This is useful when you want gradients computed in physical units (e.g., K/km instead of dimensionless):

In [4]:
# Create a mesh with coordinates in kilometers
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0),
    maxCoords=(1000.0, 2900.0),  # 1000 km × 2900 km domain
    cellSize=250.0,
    units="km",  # Coordinate units
    qdegree=2,
)

print(f"Mesh created with {mesh.data.shape[0]} nodes")
print(f"Coordinate units: {mesh.X.coords.units}")

Mesh created with 80 nodes
Coordinate units: km


In [5]:
mesh.X.coords.max()

UWQuantity(2900.0, 'kilometer')

## Working with Unit-Aware Coordinates

When a mesh has coordinate units, `mesh.X.coords` returns a `UnitAwareArray`. You can:

1. **Use `.magnitude`** to extract raw numerical values for dimensionless arithmetic
2. **Work directly with units** for unit-aware operations (multiplication, division)
3. **Perform statistics** that preserve units (max, min, mean, std)

### Method 1: Extract Raw Values with `.magnitude`

When you need to do dimensionless arithmetic (e.g., linear temperature profiles), use `.magnitude`:

In [6]:
# Create mesh variables WITH units
temperature = uw.discretisation.MeshVariable("T", mesh, 1, degree=2, units="K")
velocity = uw.discretisation.MeshVariable("u", mesh, 2, degree=2, units="m/s")

# Initialize with dimensionless arithmetic using .magnitude
with uw.synchronised_array_update():
    coords = mesh.X.coords
    x = coords[:, 0].magnitude  # Plain numpy array without units
    y = coords[:, 1].magnitude  # Plain numpy array without units

    temperature.array[...] = uw.function.evaluate(300 + 2.6 * mesh.X[1], temperature.coords)  # Linear temperature profile
    velocity.array[:, 0, 0] = 5.0  # Constant x-velocity
    velocity.array[:, 0, 1] = 0.0  # No y-velocity

print(f"Temperature units: {temperature.units}")
print(f"Temperature dimensionality: {uw.get_dimensionality(temperature)}")
print(f"Velocity units: {velocity.units}")
print(f"Velocity dimensionality: {uw.get_dimensionality(velocity)}")

Temperature units: kelvin
Temperature dimensionality: [temperature]
Velocity units: meter / second
Velocity dimensionality: [length] / [time]


In [7]:
ymax = mesh.X.coords.max()
mesh.X[1] 

N.y

In [24]:
mesh.X

In [8]:
uw.function.evaluate(mesh.X[1] / 2900 , temperature.coords)

array([[[0.        ]],

       [[0.        ]],

       [[1.        ]],

       [[1.        ]],

       [[0.        ]],

       [[0.        ]],

       [[0.        ]],

       [[1.        ]],

       [[1.        ]],

       [[1.        ]],

       [[0.08333333]],

       [[0.16666667]],

       [[0.25      ]],

       [[0.33333333]],

       [[0.41666667]],

       [[0.5       ]],

       [[0.58333333]],

       [[0.66666667]],

       [[0.75      ]],

       [[0.83333333]],

       [[0.91666667]],

       [[0.91666667]],

       [[0.83333333]],

       [[0.75      ]],

       [[0.66666667]],

       [[0.58333333]],

       [[0.5       ]],

       [[0.41666667]],

       [[0.33333333]],

       [[0.25      ]],

       [[0.16666667]],

       [[0.08333333]],

       [[0.21806233]],

       [[0.78193767]],

       [[0.62445292]],

       [[0.375     ]],

       [[0.45833333]],

       [[0.71230269]],

       [[0.28769731]],

       [[0.53265863]],

       [[0.92768376]],

       [[0.07231

### Method 2: Unit-Aware Operations

You can also work directly with units for operations like:
- **Normalization**: Dividing by a quantity with the same units
- **Unit cancellation**: Multiplying by reciprocal units
- **Statistics**: Getting max, min, mean with units preserved

In [9]:
# Get coordinates with units
coords = mesh.X.coords
y_with_units = coords[:, 1]

print(f"y coordinates type: {type(y_with_units)}")
print(f"y coordinates units: {y_with_units.units}")

# Statistics preserve units
y_max = y_with_units.max()
y_min = y_with_units.min()
y_mean = y_with_units.mean()

print(f"\nStatistics with units:")
print(f"  max: {y_max}")
print(f"  min: {y_min}")
print(f"  mean: {y_mean}")

y coordinates type: <class 'underworld3.utilities.unit_aware_array.UnitAwareArray'>
y coordinates units: km

Statistics with units:
  max: 2900.0 kilometer
  min: 0.0 kilometer
  mean: 1451.1138106694507 kilometer


In [10]:
# Unit cancellation - normalize to [0, 1]
domain_height = 2900 * uw.units.km
y_normalized = y_with_units / domain_height

print(f"Normalized y type: {type(y_normalized)}")
print(f"Normalized y range: {y_normalized.min():.3f} - {y_normalized.max():.3f}")
print(f"Note: Result is dimensionless (plain numpy array)")

Normalized y type: <class 'numpy.ndarray'>
Normalized y range: 0.000 - 1.000
Note: Result is dimensionless (plain numpy array)


In [11]:
# Alternative: Multiply by reciprocal units
y_dimensionless = y_with_units * uw.units('1/km')

print(f"Using reciprocal units:")
print(f"  Type: {type(y_dimensionless)}")
print(f"  Range: {y_dimensionless.min():.1f} - {y_dimensionless.max():.1f}")

Using reciprocal units:
  Type: <class 'numpy.ndarray'>
  Range: 0.0 - 2900.0


## Summary

### When to use `.magnitude`
- **Dimensionless arithmetic**: `temperature = 300 + 2.6 * y.magnitude`
- **Simple initialization**: When you just need raw coordinate values
- **Avoiding unit tracking**: For intermediate calculations where units would complicate things

### When to use unit-aware operations
- **Normalization**: `y / domain_height` (units cancel automatically)
- **Unit conversion**: `y * uw.units('1/cm')` (handles scale factors)
- **Statistics**: `y.max()`, `y.mean()` (preserves units in results)
- **Physical calculations**: When dimensional analysis helps catch errors

### Key Points
1. `mesh.X.coords` returns a `UnitAwareArray` when mesh has units
2. `.magnitude` extracts plain numpy arrays (no units)
3. Division by quantities with same units returns dimensionless results
4. Reduction operations (max, min, mean, std) preserve units
5. Unit cancellation happens automatically in multiplication/division

## Try It Yourself

Try these exercises:

```python
# 1. Create a mesh with different units (meters)
mesh_m = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0, 0),
    maxCoords=(100000, 50000),  # 100 km × 50 km in meters
    cellSize=5000,
    units="m",
    qdegree=2,
)

# 2. Get statistics on coordinates
x_coords = mesh_m.X.coords[:, 0]
print(f"X range: {x_coords.min()} to {x_coords.max()}")
print(f"X mean: {x_coords.mean()}")
print(f"X std dev: {x_coords.std()}")

# 3. Normalize coordinates different ways
x_norm1 = x_coords.magnitude / x_coords.max().magnitude  # Using magnitudes
x_norm2 = x_coords / (100 * uw.units.km)                # Using unit cancellation

# 4. Create a radial distance field
coords = mesh_m.X.coords
x = coords[:, 0].magnitude
y = coords[:, 1].magnitude
radius = np.sqrt(x**2 + y**2)
```